This notebook combine the preprocessed raw data and DLPs (python 3.9.18)

Libraries:
• tensorflow 2.15.0
• keras 2.15.0
• scikit-learn 1.3.0
• matplotlib 3.7.1

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from utils import *

In [2]:
DATASET_PATH = '../dataset'

### Preprocessing Pipeline 

(Result combiner, DLP + X + X_disc + y_disc)

#### 1. Origin feature wise cluster

In [3]:
# Read with pickle
with open(f'{DATASET_PATH}/dataset_sequence_cluster_1H_10W.pkl', 'rb') as f:
    data_feat_cluster = pkl.load(f)

In [4]:
indexes = []
values = None
columns = data_feat_cluster[0][0].columns
for d in data_feat_cluster:
    indexes.append(d[1].index[0])
    if values is None:
        values = np.array(d[2].values)
    else:
        values = np.concatenate([values, d[2].values])

feat_wise_x_f = pd.DataFrame(values, columns=columns, index=indexes)
feat_wise_x_f

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_4
2006-12-17 03:00:00,2,2,0,2,0,0,2,2
2006-12-17 04:00:00,2,0,0,2,0,1,2,2
2006-12-17 05:00:00,2,0,0,2,0,1,2,2
2006-12-17 06:00:00,2,1,0,2,0,1,2,2
2006-12-17 07:00:00,1,1,0,1,0,1,2,1
...,...,...,...,...,...,...,...,...
2010-11-26 17:00:00,2,0,0,2,2,1,1,2
2010-11-26 18:00:00,1,0,1,1,2,1,1,0
2010-11-26 19:00:00,1,1,1,1,2,1,2,0
2010-11-26 20:00:00,1,1,1,1,0,1,2,0


#### 2. Autoencoder Feature wise cluster

In [13]:
with open(f'{DATASET_PATH}/z_train_l1e3_cluster_dic.pkl', 'rb') as f:
    data_ae_cluster_train = pkl.load(f)
with open(f'{DATASET_PATH}/z_test_l1e3_cluster_dic.pkl', 'rb') as f:
    data_ae_cluster_test = pkl.load(f)
    
feat_wise_ae_f_train = pd.DataFrame(data_ae_cluster_train, index=indexes[:len(data_ae_cluster_train['f_z_2'])])
feat_wise_ae_f_test = pd.DataFrame(data_ae_cluster_test, index=indexes[:len(data_ae_cluster_test['f_z_2'])])
feat_wise_ae_f_train, feat_wise_ae_f_test

(                     f_z_2  f_z_4  f_z_7  f_z_13
 2006-12-17 03:00:00      0      1      0       0
 2006-12-17 04:00:00      0      0      0       0
 2006-12-17 05:00:00      0      0      0       0
 2006-12-17 06:00:00      0      0      0       0
 2006-12-17 07:00:00      0      1      0       0
 ...                    ...    ...    ...     ...
 2010-02-11 13:00:00      0      0      1       1
 2010-02-11 14:00:00      0      1      1       0
 2010-02-11 15:00:00      1      1      1       0
 2010-02-11 16:00:00      1      2      1       0
 2010-02-11 17:00:00      2      1      1       0
 
 [27663 rows x 4 columns],
                      f_z_2  f_z_4  f_z_7  f_z_13
 2006-12-17 03:00:00      2      0      1       1
 2006-12-17 04:00:00      2      0      0       2
 2006-12-17 05:00:00      1      0      0       2
 2006-12-17 06:00:00      0      0      0       1
 2006-12-17 07:00:00      0      1      1       0
 ...                    ...    ...    ...     ...
 2007-10-01 02:00:00 

In [17]:
y_hat_labels_raw = np.load(f'{DATASET_PATH}/peak_labels_hat_1008_1H_10W.npy')
y_labels_raw = np.load(f'{DATASET_PATH}/peak_labels_1H_10W.npy')
y_hat_labels_raw.shape == y_labels_raw.shape

True

In [18]:
y_hat_labels = pd.DataFrame({'y_hat_labels': y_hat_labels_raw, 
                             'y_labels': y_labels_raw}, index=indexes)

#### 3. Origin data

In [19]:
dataset_origin = []
for data in data_feat_cluster:
    dataset_origin.append((data[0],) + (data[1],))

#### 4. Combine data (x_i_w)

In [59]:
# data x_i_w_train
x_i_w_train = pd.merge(feat_wise_x_f, feat_wise_ae_f_train, left_index=True, right_index=True)
x_i_w_train = pd.merge(x_i_w_train, y_hat_labels, left_index=True, right_index=True)
x_i_w_train

# data x_i_w_test
x_i_w_test = pd.merge(feat_wise_x_f, feat_wise_ae_f_test, left_index=True, right_index=True)
x_i_w_test = pd.merge(x_i_w_test, y_hat_labels, left_index=True, right_index=True)
x_i_w_test

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_4,f_z_2,f_z_4,f_z_7,f_z_13,y_hat_labels,y_labels
2006-12-17 03:00:00,2,2,0,2,0,0,2,2,2,0,1,1,0,0
2006-12-17 04:00:00,2,0,0,2,0,1,2,2,2,0,0,2,0,0
2006-12-17 05:00:00,2,0,0,2,0,1,2,2,1,0,0,2,0,0
2006-12-17 06:00:00,2,1,0,2,0,1,2,2,0,0,0,1,0,0
2006-12-17 07:00:00,1,1,0,1,0,1,2,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-10-01 02:00:00,1,2,1,1,0,1,1,1,2,0,0,1,-1,0
2007-10-01 03:00:00,1,2,0,1,0,1,1,2,2,0,0,0,-1,-1
2007-10-01 04:00:00,2,2,0,2,0,1,1,2,2,0,0,0,0,0
2007-10-01 05:00:00,2,2,0,2,0,1,2,2,2,0,0,0,0,0


In [60]:
x_i_w_concat = pd.concat([x_i_w_train, x_i_w_test])

x_i_w_train.to_csv(f'{DATASET_PATH}/x_i_w_train.csv')
x_i_w_test.to_csv(f'{DATASET_PATH}/x_i_w_test.csv')
x_i_w_concat.to_csv(f'{DATASET_PATH}/x_i_w_combined.csv')

In [62]:
label_dic = {
    -1: 'lower',
    0: 'nochange',
    1: 'peak',
}
x_i_w_train['y_label'] = x_i_w_train.apply(lambda x: label_dic[x.y_labels], axis=1)
x_i_w_train['y_hat_label'] = x_i_w_train.apply(lambda x: label_dic[x.y_hat_labels], axis=1)

x_i_w_test['y_label'] = x_i_w_test.apply(lambda x: label_dic[x.y_labels], axis=1)
x_i_w_test['y_hat_label'] = x_i_w_test.apply(lambda x: label_dic[x.y_hat_labels], axis=1)

In [70]:
pd.get_dummies(x_i_w_train, columns=['y_hat_label', 'y_label']).to_csv(f'{DATASET_PATH}/x_i_w_train_encoded.csv', index=False)
pd.get_dummies(x_i_w_test, columns=['y_hat_label', 'y_label']).to_csv(f'{DATASET_PATH}/x_i_w_test_encoded.csv', index=False)

In [69]:
pd.get_dummies(x_i_w_train, columns=['y_hat_label', 'y_label'])

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_4,f_z_2,f_z_4,f_z_7,f_z_13,y_hat_labels,y_labels,y_hat_label_lower,y_hat_label_nochange,y_hat_label_peak,y_label_lower,y_label_nochange,y_label_peak
2006-12-17 03:00:00,2,2,0,2,0,0,2,2,0,1,0,0,0,0,False,True,False,False,True,False
2006-12-17 04:00:00,2,0,0,2,0,1,2,2,0,0,0,0,0,0,False,True,False,False,True,False
2006-12-17 05:00:00,2,0,0,2,0,1,2,2,0,0,0,0,0,0,False,True,False,False,True,False
2006-12-17 06:00:00,2,1,0,2,0,1,2,2,0,0,0,0,0,0,False,True,False,False,True,False
2006-12-17 07:00:00,1,1,0,1,0,1,2,1,0,1,0,0,0,0,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-02-11 13:00:00,0,1,2,0,0,1,0,1,0,0,1,1,0,0,False,True,False,False,True,False
2010-02-11 14:00:00,1,1,1,1,0,1,1,1,0,1,1,0,0,-1,False,True,False,True,False,False
2010-02-11 15:00:00,1,1,1,1,0,1,1,2,1,1,1,0,0,-1,False,True,False,True,False,False
2010-02-11 16:00:00,1,1,0,1,0,1,1,2,1,2,1,0,0,0,False,True,False,False,True,False
